# Package: backtesting
- References:
    - Official website: https://kernc.github.io/backtesting.py/doc/backtesting/#gsc.tab=0
    - Quick starter: https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html


In [ ]:
%%capture

!pip install --upgrade pandas-datareader
!pip install ffn backtesting

In [1]:
import ffn
from backtesting import Backtest, Strategy

## Example

In [2]:
from backtesting.test import GOOG

GOOG

c:\Users\eric.li\Desktop\My Github\Finance_programming\.venv\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
c:\Users\eric.li\Desktop\My Github\Finance_programming\.venv\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [ ]:
from backtesting.test import SMA
from backtesting.lib import crossover

class SmaCross(Strategy):    

    n1 = 5
    n2 = 10

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, self.n1)
        self.ma2 = self.I(SMA, price, self.n2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

In [ ]:
bt = Backtest(GOOG, SmaCross, cash = 10000, commission = .005, exclusive_orders = True)
stats = bt.run()
bt.plot()

In [ ]:
stats

- Glossary
    - Sortino ratio: https://en.wikipedia.org/wiki/Sortino_ratio
    - Calmar ratio: https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/calmar-ratio/
<center>
<img src = "https://cdn.corporatefinanceinstitute.com/assets/calmar-ratio1.png" width = "300px"/>
</center>
    - SQN (System Quality Number): https://www.bituzi.com/2013/11/SystemQualityNumber.html
<center>
<img src = "https://3.bp.blogspot.com/-iKaU7j_yMjo/UoTQDVqufmI/AAAAAAAAAq8/dA1EkBZ07X4/s400/2013-11-14_203502.png" width = "300px"/>
</center>

In [ ]:
stats["_trades"]

In [ ]:
stats = bt.optimize(n1 = range(5, 60, 5), n2 = range(10, 240, 10), maximize = "Equity Final [$]",
           constraint = lambda param: param.n1 < param.n2)
stats._strategy

In [ ]:
stats

## Exercise

In [ ]:
tsmc = ffn.get("2330.tw:Open, 2330.tw:High, 2330.tw:Low, 2330.tw:Close", start = "2021-01-01")
tsmc.columns = list(map(lambda x : x[6:].title(), tsmc.columns))
tsmc

In [ ]:
class SmaCross(Strategy):

    n1 = 5
    n2 = 10

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, self.n1)
        self.ma2 = self.I(SMA, price, self.n2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.position.close()
        elif crossover(self.ma2, self.ma1):
            self.sell()

In [ ]:
bt = Backtest(tsmc, SmaCross, commission = .005, cash = 100000, exclusive_orders = True)
stats = bt.run()
bt.plot()

In [ ]:
stats

In [ ]:
stats = bt.optimize(n1 = range(5, 60, 5), n2 = range(10, 240, 10), maximize = 'Sharpe Ratio',
           constraint = lambda param: param.n1 < param.n2)
stats._strategy

In [ ]:
stats